In [47]:
import numpy as np
import pandas as pd
import Data.EvalScript.evalResult as evalResult

BASE = './Data/'
START = 'START'
STOP = 'STOP'

In [48]:
class UnlabelledData:
    def __init__(self, path):
        self.sentences = []
        with open(path, 'r') as f:
            current_sentence = []
            for line in f:
                line = line.strip()
                if line == '':
                    self.sentences.append(current_sentence)
                    current_sentence = []
                else:
                    current_sentence.append(line)
            if len(current_sentence):
                self.sentences.append(current_sentence)

In [49]:
class LabelledData:
    def __init__(self, path = None):
        self.sentences = []
        if path == None:
            return
        with open(path, 'r') as f:
            current_sentence = []
            for line in f:
                line = line.strip()
                if line == '':
                    self.sentences.append(current_sentence)
                    current_sentence = []
                else:
                    current_sentence.append(tuple(line.rsplit(maxsplit=1)))
            if len(current_sentence):
                self.sentences.append(current_sentence)
    
    def write_to_file(self, path):
        with open(path, 'w') as f:
            for sentence in self.sentences:
                for data in sentence:
                    print(*data, file=f)
                print(file=f)

In [50]:
class HMModel1:
    labels = ['O', 'B-negative', 'I-negative', 'B-neutral', 'I-neutral', 'B-positive', 'I-positive']
    
    def __init__(self, k):
        self.emit_counts = {}
        self.label_counts = dict.fromkeys(self.labels, 0)
        self.k = k
    
    def get_emission_prob(self, x, y):
        return np.log(float(self.emit_counts.get((y, x), self.k))) - np.log(float((self.label_counts[y] + self.k)))
    
    def learn(self, data: LabelledData):
        for sentence in data.sentences:
            for x, y in sentence:
                for label in self.labels:
                    self.emit_counts.setdefault((label, x), 0)
                self.emit_counts[y, x] += 1
                self.label_counts[y] += 1
    
    def label(self, data: UnlabelledData):
        labelled = LabelledData()
        for unlabelled_sentence in data.sentences:
            sentence = []
            for token in unlabelled_sentence:
                y_max = None; y_max_prob = float('-inf')
                for y in self.labels:
                    y_prob = self.get_emission_prob(token, y)
                    if y_prob > y_max_prob:
                        y_max_prob = y_prob
                        y_max = y

                sentence.append((token, y_max))
            labelled.sentences.append(sentence)
        return labelled
            

In [51]:
%%time

for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')
    model = HMModel1(k=1)
    model.learn(train)
    predicted = model.label(dev_in)
    predicted.write_to_file(BASE + dataset + '/dev.p1.out')
    
    print(f'{f" {dataset} ":=^30}')
    evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p1.out')
    print('='*30)

/tmp/ipykernel_660995/589610267.py:10: RuntimeWarning: divide by zero encountered in log
  return np.log(float(self.emit_counts.get((y, x), self.k))) - np.log(float((self.label_counts[y] + self.k)))


============= RU =============
#Entity in gold data: 389
#Entity in prediction: 1816

#Correct Entity : 266
Entity  precision: 0.1465
Entity  recall: 0.6838
Entity  F: 0.2413

#Correct Sentiment : 129
Sentiment  precision: 0.0710
Sentiment  recall: 0.3316
Sentiment  F: 0.1170
============= ES =============
#Entity in gold data: 229
#Entity in prediction: 1466

#Correct Entity : 178
Entity  precision: 0.1214
Entity  recall: 0.7773
Entity  F: 0.2100

#Correct Sentiment : 97
Sentiment  precision: 0.0662
Sentiment  recall: 0.4236
Sentiment  F: 0.1145
CPU times: user 658 ms, sys: 99 µs, total: 658 ms
Wall time: 657 ms


In [52]:
class HMModel2:
    labels = ['O', 'B-negative', 'I-negative', 'B-neutral', 'I-neutral', 'B-positive', 'I-positive', START, STOP]
    
    def __init__(self, k):
        self.emit_counts = {}
        self.label_counts = dict.fromkeys(self.labels, 0)
        self.transition_counts = {}
        self.k = k
        self.label_to_index = dict(map(lambda x: (x[1], x[0]), enumerate(self.labels)))
    
    def get_emission_prob(self, x, y):
        return np.log(float(self.emit_counts.get((y, x), self.k))) - np.log(float((self.label_counts[y] + self.k)))
    
    def get_transition_prob(self, y1, y2):
        return np.log(float(self.transition_counts.get((y1, y2), 0))) - np.log(float((self.label_counts.get(y1, 1))))
    
    def learn(self, data: LabelledData):
        for sentence in data.sentences:
            prev_y = START
            for x, y in sentence:
                for label in self.labels:
                    self.emit_counts.setdefault((label, x), 0)

                self.emit_counts[y, x] += 1
                self.label_counts[y] += 1
                self.transition_counts[prev_y, y] = self.transition_counts.get((prev_y, y), 0) + 1
                prev_y = y
            self.transition_counts[prev_y, STOP] = self.transition_counts.get((prev_y, STOP), 0) + 1

            self.label_counts[START] += 1
            self.label_counts[STOP] += 1
    
    def viterbi(self, sentence, transition_prob):
        pi = np.full((len(sentence) + 1, len(self.labels)), -np.inf, dtype=np.double)
        pi[0, self.label_to_index[START]] = 0.0 # = log(1)

        for k in range(1, len(sentence) + 1):
            token = sentence[k-1]
            for vi, v in enumerate(self.labels):
                possible_next = pi[k-1, :] + transition_prob[:, vi].T + self.get_emission_prob(token, v)
                pi[k, vi] = np.max(possible_next)

        return pi

    def label(self, data):
        labelled = LabelledData()
        transition_prob = np.ndarray(shape=(len(self.labels), len(self.labels)), dtype=np.double)
        
        for ui, u in enumerate(self.labels):
            for vi, v in enumerate(self.labels):
                transition_prob[ui, vi] = self.get_transition_prob(u, v)
        
        for unlabelled_sentence in data.sentences:
            pi = self.viterbi(unlabelled_sentence, transition_prob)
            sentence = list(unlabelled_sentence)
            next_yi = self.label_to_index[STOP]
            for i in reversed(range(len(unlabelled_sentence))):
                next_yi = np.argmax(pi[i+1,:] + transition_prob[:, next_yi].T)
                sentence[i] = (sentence[i], self.labels[next_yi])
            labelled.sentences.append(sentence)
        return labelled

In [53]:
%%time

for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')
    model = HMModel2(k=1)
    model.learn(train)
    predicted = model.label(dev_in)
    predicted.write_to_file(BASE + dataset + '/dev.p2.out')
    
    print(f'{f" {dataset} ":=^30}')
    evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p2.out')
    print('='*30)

/tmp/ipykernel_660995/847748355.py:15: RuntimeWarning: divide by zero encountered in log
  return np.log(float(self.transition_counts.get((y1, y2), 0))) - np.log(float((self.label_counts.get(y1, 1))))
/tmp/ipykernel_660995/847748355.py:12: RuntimeWarning: divide by zero encountered in log
  return np.log(float(self.emit_counts.get((y, x), self.k))) - np.log(float((self.label_counts[y] + self.k)))


============= RU =============
#Entity in gold data: 389
#Entity in prediction: 488

#Correct Entity : 189
Entity  precision: 0.3873
Entity  recall: 0.4859
Entity  F: 0.4310

#Correct Sentiment : 129
Sentiment  precision: 0.2643
Sentiment  recall: 0.3316
Sentiment  F: 0.2942
============= ES =============
#Entity in gold data: 229
#Entity in prediction: 542

#Correct Entity : 134
Entity  precision: 0.2472
Entity  recall: 0.5852
Entity  F: 0.3476

#Correct Sentiment : 97
Sentiment  precision: 0.1790
Sentiment  recall: 0.4236
Sentiment  F: 0.2516
CPU times: user 1.74 s, sys: 16.1 ms, total: 1.76 s
Wall time: 1.76 s


# Part 3

Note here that $k$ has been renamed to $t$ to remove the conflict between the $k$ variable in Viterbi's algorithm.

In [64]:
class HMModel3:
    labels = ['O', 'B-negative', 'I-negative', 'B-neutral', 'I-neutral', 'B-positive', 'I-positive', START, STOP]
    
    def __init__(self, k):
        self.emit_counts = {}
        self.label_counts = dict.fromkeys(self.labels, 0)
        self.transition_counts = {}
        self.k = k
        self.label_to_index = dict(map(lambda x: (x[1], x[0]), enumerate(self.labels)))
    
    def get_emission_prob(self, x, y):
        return np.log(float(self.emit_counts.get((y, x), self.k))) - np.log(float((self.label_counts[y] + self.k)))
    
    def get_transition_prob(self, y1, y2):
        return np.log(float(self.transition_counts.get((y1, y2), 0))) - np.log(float((self.label_counts.get(y1, 1))))
    
    def learn(self, data: LabelledData):
        for sentence in data.sentences:
            prev_y = START
            for x, y in sentence:
                for label in self.labels:
                    self.emit_counts.setdefault((label, x), 0)

                self.emit_counts[y, x] += 1
                self.label_counts[y] += 1
                self.transition_counts[prev_y, y] = self.transition_counts.get((prev_y, y), 0) + 1
                prev_y = y
            self.transition_counts[prev_y, STOP] = self.transition_counts.get((prev_y, STOP), 0) + 1

            self.label_counts[START] += 1
            self.label_counts[STOP] += 1
    
    def viterbi(self, sentence, transition_prob, t):
        pi = np.full((len(sentence) + 2, len(self.labels), t), -np.inf, dtype=np.double)
        pi[0, self.label_to_index[START], 0] = 0.0 # = log(1)
        prev = np.full((len(sentence) + 1, len(self.labels), t, 2), -1, dtype=np.int64)

        for k in range(1, len(sentence) + 1):
            token = sentence[k-1]
            for vi, v in enumerate(self.labels):
                for i in range(t):
                    possible_next = pi[k-1, :, i] + transition_prob[:, vi].T + self.get_emission_prob(token, v)
                    
                    possible_prev = np.ndarray((t-i, 2), dtype=np.int64)
                    possible_prev[:, 0] = float(i)
                    possible_prev[:, 1] = np.argsort(-possible_next)[:t-i]
                    possible_next[::-1].sort()
                    cmp = possible_next[:t-i] > pi[k, vi, i:]
                    pi[k, vi, i:] = np.where(cmp, possible_next[:t-i], pi[k, vi, i:])
                    prev[k-1, vi, i:, 0] = np.where(cmp, possible_prev[:, 0], prev[k-1, vi, i:, 0])
                    prev[k-1, vi, i:, 1] = np.where(cmp, possible_prev[:, 1], prev[k-1, vi, i:, 1])
            
        for i in range(t):
            vi = self.label_to_index[STOP]
            k = len(sentence) + 1
            possible_next = pi[k-1, :, i] + transition_prob[:, vi].T

            possible_prev = np.ndarray((t-i, 2), dtype=np.int64)
            possible_prev[:, 0] = float(i)
            possible_prev[:, 1] = np.argsort(-possible_next)[:t-i]
            possible_next[::-1].sort()
            cmp = possible_next[:t-i] > pi[k, vi, i:]
            pi[k, vi, i:] = np.where(cmp, possible_next[:t-i], pi[k, vi, i:])
            prev[k-1, vi, i:, 0] = np.where(cmp, possible_prev[:, 0], prev[k-1, vi, i:, 0])
            prev[k-1, vi, i:, 1] = np.where(cmp, possible_prev[:, 0], prev[k-1, vi, i:, 1])


        return prev

    def label(self, data, t):
        labelled = LabelledData()
        transition_prob = np.ndarray(shape=(len(self.labels), len(self.labels)), dtype=np.double)
        
        for ui, u in enumerate(self.labels):
            for vi, v in enumerate(self.labels):
                transition_prob[ui, vi] = self.get_transition_prob(u, v)
        
        for unlabelled_sentence in data.sentences:
            prev = self.viterbi(unlabelled_sentence, transition_prob, t)
              

            sentence = list(unlabelled_sentence)
            next_yi = self.label_to_index[STOP]
            j = t-1
            
            for i in reversed(range(len(unlabelled_sentence))):
                p = (j, self.labels[next_yi])
                j, next_yi = prev[i+1, next_yi, j, :]
#                 if j == -1 or next_yi == -1:
#                     print("ERROR", i, '-', p, '->', (j, next_yi), '(', len(unlabelled_sentence), ')')
                sentence[i] = (sentence[i], self.labels[next_yi])
            labelled.sentences.append(sentence)
        return labelled

In [67]:
%%time

for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')

    model = HMModel3(k=2)
    model.learn(train)

    predicted2 = model.label(dev_in, 2)
    predicted2.write_to_file(BASE + dataset + '/dev.p3.2nd.out')    
    print(f'{f" {dataset} k=2 ":=^30}')
    evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p3.2nd.out')
    print('='*30)
    
    predicted8 = model.label(dev_in, 8)
    predicted8.write_to_file(BASE + dataset + '/dev.p3.8th.out')
    print(f'{f" {dataset}  k=8 ":=^30}')
    evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p3.8th.out')
    print('='*30)

/tmp/ipykernel_660995/959558809.py:15: RuntimeWarning: divide by zero encountered in log
  return np.log(float(self.transition_counts.get((y1, y2), 0))) - np.log(float((self.label_counts.get(y1, 1))))
/tmp/ipykernel_660995/959558809.py:12: RuntimeWarning: divide by zero encountered in log
  return np.log(float(self.emit_counts.get((y, x), self.k))) - np.log(float((self.label_counts[y] + self.k)))


=========== RU k=2 ===========
#Entity in gold data: 389
#Entity in prediction: 772

#Correct Entity : 106
Entity  precision: 0.1373
Entity  recall: 0.2725
Entity  F: 0.1826

#Correct Sentiment : 61
Sentiment  precision: 0.0790
Sentiment  recall: 0.1568
Sentiment  F: 0.1051
========== RU  k=8 ===========
#Entity in gold data: 389
#Entity in prediction: 332

#Correct Entity : 60
Entity  precision: 0.1807
Entity  recall: 0.1542
Entity  F: 0.1664

#Correct Sentiment : 28
Sentiment  precision: 0.0843
Sentiment  recall: 0.0720
Sentiment  F: 0.0777
=========== ES k=2 ===========
#Entity in gold data: 229
#Entity in prediction: 475

#Correct Entity : 47
Entity  precision: 0.0989
Entity  recall: 0.2052
Entity  F: 0.1335

#Correct Sentiment : 26
Sentiment  precision: 0.0547
Sentiment  recall: 0.1135
Sentiment  F: 0.0739
========== ES  k=8 ===========
#Entity in gold data: 229
#Entity in prediction: 232

#Correct Entity : 42
Entity  precision: 0.1810
Entity  recall: 0.1834
Entity  F: 0.1822

#Co